.

## Detection and Prevention of Vehicle Insurance Claim Fraud

### Muhammad Haris, Aaron Ramirez, and Outhai Xayavongsa

Vehicle insurance fraud is a major issue that results in significant financial losses for insurance companies and undermines trust in the insurance system. Fraudulent claims can range from staged accidents to exaggerated personal injury claims, complicating the claims process and driving up costs (Punith, 2021). Our key question is: <span>How can vehicle and policy data be leveraged to accurately detect and prevent fraudulent insurance claims?</span> Our primary goal is to develop a reliable predictive model that identifies fraudulent claims using historical data. This model aims to help insurance companies reduce financial losses, streamline claims processing, and ensure fair premium pricing for customers.

Our approach to detecting and preventing vehicle insurance claim fraud involves several key steps, which will be meticulously documented and reviewed to ensure technical soundness and business viability:
* **Data Integrity:** Examine the data structure, check for missing values, and convert data types appropriately. This step ensures the foundation of our dataset is solid and ready for analysis.
* **Encoding and Scaling:** Convert categorical variables into numerical formats using one-hot encoding and standardize numerical features. This step is crucial for preparing the data for machine learning algorithms.
* **Exploratory Data Analysis (EDA):** Analyze the target variable distribution, visualize numerical and categorical features, and perform correlation analysis. This provides insights into the data and helps identify patterns.
* **Feature Selection:** Set a correlation threshold, use a Random Forest classifier to determine feature importance, and select the most predictive features. This step helps in identifying the most relevant features for our model.
* **Feature Engineering:**  Enhance the dataset with interaction and polynomial features. This can potentially improve the model's ability to detect complex patterns indicative of fraud.
* **Modeling and Evaluation:** 
  1. Baseline Models: Train baseline models and evaluate their performance to establish a benchmark.
  2. Hyperparameter Tuning: Optimize the models through hyperparameter tuning to enhance performance.
  3. Model Selection: Select the best-performing model based on evaluation metrics.
  4. Ensemble Methods: Combine multiple models to improve accuracy and robustness.
  5. Final Model Training: Train the final model with selected features and prepare it for deployment.<br><br>
  
* **Deployment:** Deploy the model in a production environment and continuously monitor its performance. This step ensures the model is operational and provides real-time fraud detection.
* **Discussion and Conclusion:** Provide actionable recommendations based on the model's findings. Discuss business, ethical, and regulatory risks associated with the deployment and use of the model.

### Understanding the Data
To ensure data integrity and suitability for model training, it's essential to comprehend the data's structure, types, and initial statistics.

In [ ]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import scipy.stats as stats  # Ensure this line is present
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import chi2_contingency
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Load the Dataset
file = 'fraud_oracle.csv'
df = pd.read_csv(file)

# Display all columns
pd.set_option('display.max_columns', None)

# Display first few rows of the dataset
df.head()

In [ ]:
# Descriptive Statistics
df.describe(include='all')

### Data Cleaning

**Checking Missing Values** is crucial as they can impact the performance of machine learning models. Depending on the extent of missing data, different strategies (imputation, removal) may be employed.

In [ ]:
# Check Missing Values
print("Missing Values in each column:")
df.isna().sum()

In [ ]:
# Display basic information about the dataset
print(df.dtypes)

### Converting Data Types
Ensuring correct data types facilitates proper handling during preprocessing and analysis.

In [ ]:
# Data Type Conversion
df['FraudFound_P'] = df['FraudFound_P'].astype(bool)
df['Year'] = df['Year'].astype(object)

#print(df['FraudFound_P'].dtypes)
#print(df['Year'].dtypes)

# Extract numerical and categorical columns
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

In [ ]:
# Convert categorical columns to category dtype
df[categorical_cols] = df[categorical_cols].astype('category')

# Display the first few rows again to confirm changes
df.head()

## Exploratory Data Analysis (EDA)

Analyze the target variable distribution, visualize numerical and categorical features, and perform correlation analysis. This provides insights into the data and helps identify patterns.

**Distribution of Target Variable**<br>

In [ ]:
# Count the occurrences of each category in FraudFound_P
fraud_counts = df['FraudFound_P'].value_counts()

# Plot the pie chart
plt.figure(figsize=(5, 5))  # Increase the figure size for better readability
plt.pie(fraud_counts, labels=fraud_counts.index, autopct='%1.1f%%', startangle=140, colors=['lightblue', 'orange'])
plt.title('Distribution of Fraudulent Claims (FraudFound_P)', fontsize=12)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.legend(title='Fraud Found', labels=['Not Fraud', 'Fraud'], loc='upper right', bbox_to_anchor=(1.2, 1))
plt.show()

<span>The pie chart shows that 94% of the claims are not fraudulent, while only **6%** are identified as **fraudulent.** This indicates that fraudulent claims make up a small portion of the overall claims. It underscores the challenge of fraud detection due to the rarity of fraudulent cases compared to the overwhelming number of legitimate claims.</span>

**Numerical Feature Analysis**<br>
Visualize the distribution of numerical features and their relationship with the target variable.

In [ ]:
# Plot histograms for numerical features
num_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Plotting histograms with larger labels for x and y axis
plt.figure(figsize=(20, 20))
for i, col in enumerate(num_cols):
    plt.subplot((len(num_cols) // 3) + 1, 3, i + 1)
    plt.hist(df[col], bins=20, edgecolor='black')
    plt.title(f'Histogram of {col}', fontsize=16)
    plt.xlabel(col, fontsize=14)
    plt.ylabel('Frequency', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

plt.tight_layout()
plt.show()

<span>From these histograms, several patterns emerge that might impact fraud detection. </span>
* <span>The distribution of **Ag**e is right-skewed, with a higher concentration of claims from individuals aged 30 to 50, suggesting that this age group may be more susceptible to fraudulent activities. </span>
* <span>The **PolicyNumber** shows a uniform distribution, indicating no specific policy sequence associated with fraud. </span>
* <span>The **Deductible** distribution reveals a concentration at lower values, specifically around 400, which could suggest that lower deductible policies might be more prone to fraudulent claims. </span>
* <span>**RepNumber** indicates some variation, which may hint at certain representatives having higher fraud incidents. </span>
* <span>**DriverRating** is evenly distributed, showing no significant impact on fraud. </span>
* <span>**WeekOfMonth** and **WeekOfMonthClaimed** both show consistent distributions, implying that the timing within a month does not significantly affect fraud occurrence. </span>

<span>These insights underscore the need for a nuanced approach to fraud detection, leveraging advanced machine learning techniques to identify subtle and complex patterns.

**Target-based Feature Distribution**<br>
It is crucial for feature selection and engineering as it relates the features to the problem of fraud detection.

In [ ]:
# Plot distribution of numerical features by FraudFound_P
plt.figure(figsize=(20, 20))  # Increase the height of the figure
num_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Adjusting the number of plots per row
plots_per_row = 2

for i, col in enumerate(num_cols):
    plt.subplot((len(num_cols) // plots_per_row) + 1, plots_per_row, i + 1)
    sns.histplot(data=df, x=col, hue='FraudFound_P', kde=True, element='step')
    plt.title(f'Distribution of {col} by FraudFound_P')
    plt.xlabel(col)
    plt.ylabel('Count')

plt.tight_layout()
plt.show()

<span>These visualizations highlight a few key insights regarding fraudulent claims. </span>
* <span>Notably, the distribution of fraudulent claims shows a slight concentration in the 30-50 **age** range, suggesting individuals within this bracket might be more prone to fraud. On the other hand, The histogram of age distribution in fraudulent claims indicates a significant spike in the 25-35 age range. This pattern aligns with findings from a survey by Verisk and the Coalition Against Insurance Fraud, which suggests that younger generations (ages 18-44) exhibit a higher tolerance and willingness to engage in insurance fraud compared to older generations (MehaffyWeber, 2023).</span>
* <span>The **PolicyNumber** and **Deductible** distributions indicate no particular sequence pattern or deductible amount that differentiates fraudulent claims, as both are uniformly distributed. </span>
* <span>Additionally, the **RepNumber** distribution hints at potential biases or internal issues, as some representatives may have higher incidences of fraud. </span>
* <span>The **DriverRating** distribution is even, indicating it is not a strong predictor of fraud. </span><br>

<span>Overall, these observations suggest that fraudulent claims are spread across various features, underscoring the need for advanced modeling techniques to detect complex interactions and subtle patterns effectively. Both analyses are necessary. The first set provides a foundational understanding of the data, while the second set builds on this by showing how the features interact with the target variable. Together, they offer a comprehensive view that is essential for effective feature engineering and selection in machine learning workflows.</span>

The **boxplots** compare the distribution of numerical features between fraudulent and non-fraudulent claims.

In [ ]:
# Plot boxplots to check for outliers and their relation with the target variable
plt.figure(figsize=(15, 20))
for i, col in enumerate(num_cols):
    plt.subplot(4, 2, i+1)
    sns.boxplot(x='FraudFound_P', y=col, data=df)
    plt.title(f'{col}')
plt.tight_layout()
plt.show()

<span>The boxplots above compare various numerical features against the target variable, FraudFound_P. </span>
* <span>The boxplots for **"WeekOfMonth"** and **"WeekOfMonthClaimed"** show a similar distribution between fraudulent and non-fraudulent claims, suggesting these features may not significantly differentiate fraud. These also show no significant outliers. No removal is needed here.</span>
* <span>The **"Age"** feature shows a slight difference, with fraudulent claims tending to involve slightly older individuals, but with significant overlap. There are a few outliers, particularly for ages above 70. Eventhough, we have these outliers, they are not extreme enough that it will distort our model. It is possible to remove them if they are considered anomalies. Elderly drivers are more likely to have insurance claims due to increased accident risks stemming from health issues and stringent legal requirements (Alvendia, 2024).</span>
* <span>**"PolicyNumber"** and **"RepNumber"** also do not show distinct differences between fraud and non-fraud cases. No outliers exist. </span>
* <span>The **"Deductible"** There are a few extreme outliers below 350 and above 600. This could distort the model and should be removed. The central tendency is similar for both fraud and non-fraud. </span>
* <span>Lastly, **"DriverRating"** shows a slight increase in ratings for fraudulent claims, though the difference is minor. No outliers exist.<br><br>Overall, the boxplots indicate that many features do not strongly distinguish between fraudulent and non-fraudulent claims, with some exceptions requiring further analysis. Outliers exist in Deductible.</span>

Outlier Removal: Identify and remove outliers from the Deductible feature.

In [ ]:
# Identify outliers in the "Deductible" feature
Q1 = df['Deductible'].quantile(0.25)
Q3 = df['Deductible'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for acceptable data
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df_cleaned = df[(df['Deductible'] >= lower_bound) & (df['Deductible'] <= upper_bound)]

# Display the shape of the dataframe before and after outlier removal
print("Original dataset shape:", df.shape)
print("Dataset shape after outlier removal:", df_cleaned.shape)

<span>Since the other features do not have significant outliers or the outliers are not extreme enough to distort the model, no further outlier removal is necessary.</span>

The **chi-square test** is used to examine the relationship between each categorical feature and the target variable (FraudFound_P).

In [ ]:
# Convert categorical variables to strings to ensure consistent handling
cat_cols = df.select_dtypes(include=['object', 'category']).columns
df[cat_cols] = df[cat_cols].astype(str)
df['FraudFound_P'] = df['FraudFound_P'].astype(str)

# Calculate chi-square statistics and store in a DataFrame
chi2_results = []
for col in cat_cols:
    contingency_table = pd.crosstab(df[col], df['FraudFound_P'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    chi2_results.append({'Feature': col, 'Chi2': chi2, 'p-value': p, 'Degrees of Freedom': dof})

chi2_df = pd.DataFrame(chi2_results)
chi2_df

<span>The **chi-square test** examines the relationship between each categorical feature and the target variable (FraudFound_P). Features with low p-values indicate a significant association with fraud, providing insights for model development.</span>

<span>Results with best association to fraud:</span>
* <span>PolicyType</span>
* <span>VehicleCategory</span>
* <span>Fault</span>
* <span>BasePolicy</span>
* <span>VehiclePrice</span>
* <span>Make</span>
* <span>AddressChange_Claim</span>
* <span>PastNumberOfClaims</span>
* <span>MonthClaimed</span>
* <span>AgeOfPolicyHolder</span>
* <span>AccidentArea</span>
* <span>Days_Policy_Accident</span>
* <span>Sex</span>
* <span>NumberOfSuppliments</span>
* <span>AgentType</span>
* <span>AgeOfVehicle</span>

The **t-test** is performed to determine if there are statistically significant differences in the means of numerical features between fraudulent and non-fraudulent claims.

In [ ]:
df['FraudFound_P'] = df['FraudFound_P'].replace({'False': 0, 'True': 1})

# Convert the target variable to numerical
df['FraudFound_P'] = df['FraudFound_P'].astype(int)

# Define groups
fraudulent = df[df['FraudFound_P'] == 1]
non_fraudulent = df[df['FraudFound_P'] == 0]

# List of numerical features
numerical_features = ['WeekOfMonth', 'WeekOfMonthClaimed', 'Age', 'PolicyNumber', 'RepNumber', 'Deductible', 'DriverRating']

# Perform t-tests for each numerical feature
alpha = 0.05  # significance level
results = []

for feature in numerical_features:
    t_stat, p_val = stats.ttest_ind(fraudulent[feature].dropna(), non_fraudulent[feature].dropna())
    result = {
        'Feature': feature,
        'T-statistic': t_stat,
        'P-value': p_val,
        'Significant': p_val < alpha
    }
    results.append(result)
    # Print results
    print(f'Feature: {feature}')
    print(f'T-statistic: {t_stat}')
    print(f'P-value: {p_val}')
    if p_val < alpha:
        print("Reject the null hypothesis: There is a significant difference between the two groups.\n")
    else:
        print("Fail to reject the null hypothesis: There is no significant difference between the two groups.\n")

# Convert results to DataFrame for better visualization if needed
results_df = pd.DataFrame(results)
print(results_df)

<span>Based on the **T-Test**, the features **Age, PolicyNumber, and Deductible** show significant with the target variable.</span>

**Categorical Feature Analysis** using the bar plots involves examining the frequency distribution of categorical variables and their relationship with the target variable, FraudFound_P.

In [ ]:
# Plot bar plots for categorical features
plt.figure(figsize=(20, 30))
for i, col in enumerate(cat_cols):
    print(f"Processing feature: {col}")
    plt.subplot(8, 4, i + 1)
    try:
        sns.countplot(x=col, hue='FraudFound_P', data=df)
        
        # Extract p-value
        p_value_row = chi2_df.loc[chi2_df['Feature'] == col]
        if not p_value_row.empty:
            p_value = p_value_row['p-value'].values[0]
            plt.title(f'Distribution of {col} by FraudFound_P\n(p-value: {p_value:.4f})')
        else:
            plt.title(f'Distribution of {col} by FraudFound_P\n(p-value: N/A)')
        
        plt.xticks(rotation=90)
    except Exception as e:
        #print(f"Error processing feature {col}: {e}")
        pass
plt.tight_layout()
plt.show()

<span>Based on the **bar plot** visual analysis, the features that stand out with a noticeable difference in fraudulent activity are:

* <span>**Fault:** Fraudulent claims are significantly higher when the policyholder is at fault. This feature has a clear distinction between fraud and non-fraud cases.
    
* <span>**PolicyType:** Certain policy types, especially "Sport - Collision" and "Utility - Collision," show a markedly higher incidence of fraudulent claims.
    
* <span>**VehicleCategory:** Sports vehicles have a significantly higher number of fraudulent claims compared to other categories.
    
* <span>**BasePolicy:** 'Liability' policies are associated with a higher number of fraudulent claims compared to 'Collision' and 'All Perils' policies.

* <span>**VehiclePrice:** Mid-range vehicles (such as those priced between 30,000 to 39,000 and 40,000 to 59,000) show a higher incidence of fraud.
    
* <span>**PastNumberOfClaims:** Individuals with '2 to 4' and 'more than 4' past claims exhibit higher rates of fraudulent activity.
    
* <span>**Make:** Certain makes, such as Honda, Toyota, Mazda, and Pontiac, have higher fraudulent claim rates.
    
* <span>**Sex:** Males show a higher number of fraudulent claims compared to females.
    
* <span>**AgeOfVehicle:** Newer vehicles (3 to 4 years old) have a noticeably higher rate of fraudulent claims.
    
* <span>**AddressChange_Claim:** Policyholders with no recent address changes have more fraudulent claims, suggesting this feature is a significant indicator of fraud.

**Pair Plot** will help us visualize the pairwise relationships between features and the target variable.

In [ ]:
# Plot pair plot
sns.pairplot(df, hue='FraudFound_P', diag_kind='kde')
plt.suptitle('Pair Plot of Features')
plt.show()

<span>The **pair plot** provides a comprehensive visualization of the relationships and distributions among several numerical features in the dataset, highlighting their interaction with the target variable, FraudFound_P. Key features analyzed include WeekOfMonth, WeekOfMonthClaimed, Age, PolicyNumber, RepNumber, Deductible, and DriverRating. The distribution of WeekOfMonth and WeekOfMonthClaimed is even across different weeks, showing no clear distinction between fraudulent and non-fraudulent claims. The age distribution is right-skewed, with most claimants between 20 and 50 years old, but no strong separation based on age is observed. PolicyNumber appears uniformly distributed, indicating no particular sequence pattern that differentiates fraudulent claims, while RepNumber shows a varied distribution, which could be further explored for potential bias. The deductible amounts are concentrated at specific values, with no clear separation for fraudulent claims, and DriverRating is evenly distributed across its range, again showing no distinct separation for fraudulent claims.</span>

<span>The scatter plots in the pair plots do not reveal strong linear relationships between pairs of features, indicating that the relationships might be complex and non-linear, making machine learning models like Random Forests or XGBoost suitable. Color coding by FraudFound_P (False in light blue and True in salmon) shows no clear clusters or separations in the scatter plots, suggesting that fraudulent claims are not easily distinguishable based on simple visual inspection. This reinforces the need for sophisticated modeling techniques due to the imbalanced nature of the dataset, with fraudulent claims being relatively rare compared to non-fraudulent ones. <br></span>

Plot the **correlation matrix** for all numerical features in the dataset. This involves calculating the pairwise correlation coefficients between all numerical variables. Correlation coefficients range from -1 (negative correlation) to 1 (positive correlation)

In [ ]:
# Calculate the correlation matrix
corr_matrix = df[num_cols].corr()

# Correlation Matrix for all Numerical Features
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix for All Numerical Features with the target variable (FraudFound_P)')
plt.show()

<span>The correlation matrix for all numerical features with the target variable (FraudFound_P) indicates that none of the features have a strong correlation with fraud. The correlations are as follows:</span>

* <span>WeekOfMonth: -0.017</span>
* <span>WeekOfMonthClaimed: -0.004</span>
* <span>Age: -0.0048</span>
* <span>PolicyNumber: 0.0053</span>
* <span>RepNumber: -0.0078</span>
* <span>Deductible: 0.017</span>
* <span>DriverRating: -0.017</span>

<span>Among these features, PolicyNumber has the highest positive correlation with fraud at 0.0053, and Deductible has a similar but slightly negative correlation at -0.017. However, these values are extremely close to zero, indicating very weak or negligible relationships. Therefore, no single numerical feature strongly correlates with fraudulent claims, highlighting the need for more sophisticated methods or combined feature interactions to detect fraud effectively.</span>

<h1>Feature Engineering</h1>

### Features evidence from Domain Knowledge
Claims Involving High-Value Items
Claims involving luxury or high-value vehicles, or expensive personal items, are often scrutinized more rigorously due to the higher payout potential.
Source: Brown, M. (2020). "High-Value Claims and Fraud Risk." Journal of High-Value Insurance, 16(3), 77-88.

Claimant’s History of Suspicious Activity
Any history of suspicious behavior, such as previous investigations or reports of fraud, even if not proven, can lead insurers to scrutinize new claims more closely.
Source: Green, D. (2019). "Historical Analysis of Suspicious Activity in Insurance." Insurance Fraud Studies, 19(1), 56-67.

Unusual Claim Patterns
Claims with similar circumstances or involving the same parties repeatedly can be suspicious. For example, if a policyholder has multiple claims involving the same type of accident or damage, this might indicate an attempt to defraud the insurer.
Source: Jones, H. (2021). "Patterns of Unusual Claims in Auto Insurance Fraud." Journal of Insurance Research, 22(1), 56-68.

Additional Indicator: Claim Amounts Just Above Deductible
Insurance companies scrutinize claims where the amount is just above the policy's deductible. This pattern may suggest that claimants are attempting to maximize their payout while minimizing their out-of-pocket expenses.
Source: Thompson, R. (2022). "Deductible Thresholds and Fraudulent Claims." Insurance Fraud Insights, 20(2), 37-48.

Additional Indicator: Fault in the Accident
Insurance companies examine the details of who was at fault in the accident. If the claimant consistently claims to be not at fault in multiple incidents, it may raise suspicions of fraud.
Source: Hernandez, G. (2021). "Fault Analysis in Fraud Detection." Insurance Fraud Journal, 16(3), 29-40.

Age and Demographics
Younger drivers and those with certain demographic profiles may be statistically more likely to file fraudulent claims.
Source: Johnson, L. (2020). "Demographic Indicators in Insurance Fraud." Insurance Journal, 32(4), 67-78.

**Encode Categorical Variables and Scale Numerical Colummns**<br>
Machine learning models require numerical inputs, so we need to convert categorical variables into a numerical format using one-hot encoding.

### References still being debated
Alvendia, R. (2024, March 6). <i>Elderly safety in car accidents.</i> AKD Lawyers. https://www.akdlawyers.com/car-accident/elderly-safety-in-car-accidents/

Bansal, S. (2022). <i>Vehicle insurance claim fraud detection.</i> Kaggle.     https://www.kaggle.com/datasets/shivamb/vehicle-claim-fraud-detection

MehaffyWeber. (2023, October 3). <i>Survey finds younger generations have a higher tolerance for insurance fraud – What insurers should know.</i> MehaffyWeber. https://www.mehaffyweber.com/news/survey-finds-younger-generations-have-a-higher-tolerance-for-insurance-fraud-what-insurers-should-know/

Punith. (2021, June 24). <i>Insurance claims — Fraud detection using machine learning.</i> Geek Culture.   https://medium.com/geekculture/insurance-claims-fraud-detection-using-machine-learning-78f04913097

### Features that will be used

* age of vehicle
* age group 30 to 50 / 18 - 44 (we will do 18 - 50)
* pastnumber of claims
* vehicle price
* vehicle category
* base policy
* fault
* deductible
* policytype
* base policy
* sex

In [26]:
df.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,FraudFound_P,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,21,Policy Holder,Sport - Liability,Sport,more than 69000,0,1,12,300,1,more than 30,more than 30,none,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,34,Policy Holder,Sport - Collision,Sport,more than 69000,0,2,15,400,4,more than 30,more than 30,none,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,47,Policy Holder,Sport - Collision,Sport,more than 69000,0,3,7,400,3,more than 30,more than 30,1,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,65,Third Party,Sedan - Liability,Sport,20000 to 29000,0,4,4,400,2,more than 30,more than 30,1,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,27,Third Party,Sport - Collision,Sport,more than 69000,0,5,3,400,1,more than 30,more than 30,none,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


### Feature Extraction from EDA and Domain Knowledge

In [27]:
# Extract relevant columns
columns_of_interest = [
    'FraudFound_P',
    'AgeOfVehicle',
    'AgeOfPolicyHolder',
    'PastNumberOfClaims',
    'VehiclePrice',
    'VehicleCategory',
    'BasePolicy',
    'Fault',
    'Deductible',
    'PolicyType',
    'Sex'
]

# Check which of these columns are present in the dataframe
df_relevant = df[columns_of_interest]

# Display the first few rows of the cleaned dataframe
df_relevant.head()


,FraudFound_P,AgeOfVehicle,AgeOfPolicyHolder,PastNumberOfClaims,VehiclePrice,VehicleCategory,BasePolicy,Fault,Deductible,PolicyType,Sex
0,0,3 years,26 to 30,none,more than 69000,Sport,Liability,Policy Holder,300,Sport - Liability,Female
1,0,6 years,31 to 35,none,more than 69000,Sport,Collision,Policy Holder,400,Sport - Collision,Male
2,0,7 years,41 to 50,1,more than 69000,Sport,Collision,Policy Holder,400,Sport - Collision,Male
3,0,more than 7,51 to 65,1,20000 to 29000,Sport,Liability,Third Party,400,Sedan - Liability,Male
4,0,5 years,31 to 35,none,more than 69000,Sport,Collision,Third Party,400,Sport - Collision,Female


### One Hot Encoding
Feature binning already happened with age and income when we this was encoded

In [28]:
# Encode categorical variables
df_encoded = pd.get_dummies(df_relevant, drop_first=True)
df_encoded = df_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))
df_encoded = df_encoded.map(lambda x: 1 if x is True else (0 if x is False else x))

# Print first few rows of encoded and scaled dataset
df_encoded.head()

,FraudFound_P,Deductible,AgeOfVehicle_3 years,AgeOfVehicle_4 years,AgeOfVehicle_5 years,AgeOfVehicle_6 years,AgeOfVehicle_7 years,AgeOfVehicle_more than 7,AgeOfVehicle_new,AgeOfPolicyHolder_18 to 20,AgeOfPolicyHolder_21 to 25,AgeOfPolicyHolder_26 to 30,AgeOfPolicyHolder_31 to 35,AgeOfPolicyHolder_36 to 40,AgeOfPolicyHolder_41 to 50,AgeOfPolicyHolder_51 to 65,AgeOfPolicyHolder_over 65,PastNumberOfClaims_2 to 4,PastNumberOfClaims_more than 4,PastNumberOfClaims_none,VehiclePrice_30000 to 39000,VehiclePrice_40000 to 59000,VehiclePrice_60000 to 69000,VehiclePrice_less than 20000,VehiclePrice_more than 69000,VehicleCategory_Sport,VehicleCategory_Utility,BasePolicy_Collision,BasePolicy_Liability,Fault_Third Party,PolicyType_Sedan - Collision,PolicyType_Sedan - Liability,PolicyType_Sport - All Perils,PolicyType_Sport - Collision,PolicyType_Sport - Liability,PolicyType_Utility - All Perils,PolicyType_Utility - Collision,PolicyType_Utility - Liability,Sex_Male
0,0,300,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0
1,0,400,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,1
2,0,400,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,1
3,0,400,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1
4,0,400,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0


### Standardizing numerical features 
Ensures they have a mean of zero and a standard deviation of one, which helps many machine learning models perform better.

In [29]:
# Assuming df_encoded is your dataframe
scaler = StandardScaler()

# Reshape the data for the scaler
df_encoded['Deductible'] = scaler.fit_transform(df_encoded['Deductible'].values.reshape(-1, 1))

# Display the first few rows of the encoded and scaled dataset
df_encoded.head()

#coding note. If the car has 0 for 1995 and 1996, it is considered 1994

,FraudFound_P,Deductible,AgeOfVehicle_3 years,AgeOfVehicle_4 years,AgeOfVehicle_5 years,AgeOfVehicle_6 years,AgeOfVehicle_7 years,AgeOfVehicle_more than 7,AgeOfVehicle_new,AgeOfPolicyHolder_18 to 20,AgeOfPolicyHolder_21 to 25,AgeOfPolicyHolder_26 to 30,AgeOfPolicyHolder_31 to 35,AgeOfPolicyHolder_36 to 40,AgeOfPolicyHolder_41 to 50,AgeOfPolicyHolder_51 to 65,AgeOfPolicyHolder_over 65,PastNumberOfClaims_2 to 4,PastNumberOfClaims_more than 4,PastNumberOfClaims_none,VehiclePrice_30000 to 39000,VehiclePrice_40000 to 59000,VehiclePrice_60000 to 69000,VehiclePrice_less than 20000,VehiclePrice_more than 69000,VehicleCategory_Sport,VehicleCategory_Utility,BasePolicy_Collision,BasePolicy_Liability,Fault_Third Party,PolicyType_Sedan - Collision,PolicyType_Sedan - Liability,PolicyType_Sport - All Perils,PolicyType_Sport - Collision,PolicyType_Sport - Liability,PolicyType_Utility - All Perils,PolicyType_Utility - Collision,PolicyType_Utility - Liability,Sex_Male
0,0,-2.450633,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0
1,0,-0.175298,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,1
2,0,-0.175298,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,1
3,0,-0.175298,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1
4,0,-0.175298,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0


In [30]:
# Extract the target column
target_col = df_encoded['FraudFound_P'].copy()

#target_col.head()

In [31]:
# Assuming df_relevant is already prepared and target variable 'FraudFound_P' is included

# Separate features and target variable
X = df_encoded.drop(columns=['FraudFound_P'])
y = df_encoded['FraudFound_P']

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Combine resampled features and target into a new dataframe
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['FraudFound_P'] = y_resampled

df_resampled.head()
X.head()

,Deductible,AgeOfVehicle_3 years,AgeOfVehicle_4 years,AgeOfVehicle_5 years,AgeOfVehicle_6 years,AgeOfVehicle_7 years,AgeOfVehicle_more than 7,AgeOfVehicle_new,AgeOfPolicyHolder_18 to 20,AgeOfPolicyHolder_21 to 25,AgeOfPolicyHolder_26 to 30,AgeOfPolicyHolder_31 to 35,AgeOfPolicyHolder_36 to 40,AgeOfPolicyHolder_41 to 50,AgeOfPolicyHolder_51 to 65,AgeOfPolicyHolder_over 65,PastNumberOfClaims_2 to 4,PastNumberOfClaims_more than 4,PastNumberOfClaims_none,VehiclePrice_30000 to 39000,VehiclePrice_40000 to 59000,VehiclePrice_60000 to 69000,VehiclePrice_less than 20000,VehiclePrice_more than 69000,VehicleCategory_Sport,VehicleCategory_Utility,BasePolicy_Collision,BasePolicy_Liability,Fault_Third Party,PolicyType_Sedan - Collision,PolicyType_Sedan - Liability,PolicyType_Sport - All Perils,PolicyType_Sport - Collision,PolicyType_Sport - Liability,PolicyType_Utility - All Perils,PolicyType_Utility - Collision,PolicyType_Utility - Liability,Sex_Male
0,-2.450633,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0
1,-0.175298,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,1
2,-0.175298,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,1
3,-0.175298,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1
4,-0.175298,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0


### Splitting the Data

In [32]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_resampled.drop(columns=['FraudFound_P']), df_resampled['FraudFound_P'], test_size=0.2, random_state=42)

## Models

### Model 1: Isolation Forest

Isolation Forest is suitable for anomaly detection, which can help identify fraudulent claims. It isolates observations by randomly selecting a feature and then randomly selecting a split value between the maximum and minimum values of the selected feature.

In [34]:
# Train Isolation Forest Model
isolation_forest = IsolationForest(contamination=0.06, random_state=42)
isolation_forest.fit(X_train)
y_pred_if = isolation_forest.predict(X_test)
y_pred_if = [1 if x == -1 else 0 for x in y_pred_if]

# Evaluate Isolation Forest Model
print("Isolation Forest:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_if)}")
print(classification_report(y_test, y_pred_if))
print(confusion_matrix(y_test, y_pred_if))

Isolation Forest:
Accuracy: 0.46249353336782206
              precision    recall  f1-score   support

           0       0.48      0.90      0.63      2899
           1       0.19      0.02      0.04      2900

    accuracy                           0.46      5799
   macro avg       0.33      0.46      0.33      5799
weighted avg       0.33      0.46      0.33      5799

[[2618  281]
 [2836   64]]


The initial results show that the **Isolation Forest model** has an accuracy of 48.25%, indicating it is better at identifying legitimate claims than fraudulent ones. The confusion matrix indicates that it correctly identified 2,678 legitimate claims but failed to identify a significant number of fraudulent claims (2,780).
<table border="1" style="text-align: left;">
  <tr>
    <th></th>
    <th>Predicted Legitimate</th>
    <th>Predicted Fraudulent</th>
  </tr>
  <tr>
    <td><strong>Actual Legitimate</strong></td>
    <td>2678 (True Negatives)</td>
    <td>221 (False Positives)</td>
  </tr>
  <tr>
    <td><strong>Actual Fraudulent</strong></td>
    <td>2780 (False Negatives)</td>
    <td>120 (True Positives)</td>
  </tr>
</table>

### Hyperparameter Tuning for Isolation Forest Model

Tuning the Isolation Forest model involves adjusting parameters like the contamination rate and the number of estimators to better identify fraudulent claims. This fine-tuning helps improve the model's ability to detect anomalies by fitting the data more accurately.

In [36]:
# Define a custom scoring function that fits the nature of the problem
def anomaly_score(y_true, y_pred):
    y_pred = [1 if x == -1 else 0 for x in y_pred]
    return f1_score(y_true, y_pred)

# Create the scorer
scorer = make_scorer(anomaly_score, greater_is_better=True)

# Define the parameter grid
param_distributions_if = {
    'n_estimators': [50, 100, 150],
    'max_samples': ['auto', 0.8, 0.9],
    'contamination': [0.05, 0.06, 0.07],
    'max_features': [1.0, 0.8, 0.9]
}

# Initialize RandomizedSearchCV
random_search_if = RandomizedSearchCV(IsolationForest(random_state=42), param_distributions_if, n_iter=20, cv=5, scoring=scorer, random_state=42, n_jobs=-1)

# Fit to the training data
random_search_if.fit(X_train, y_train)  # Note: y_train is used here to compute the score, not for training the model

# Best parameters and evaluation
print("Best parameters for Isolation Forest:", random_search_if.best_params_)
best_if = random_search_if.best_estimator_
y_pred_if_best = best_if.predict(X_test)
y_pred_if_best = [1 if x == -1 else 0 for x in y_pred_if_best]

print("Tuned Isolation Forest:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_if_best)}")
print(classification_report(y_test, y_pred_if_best))
print(confusion_matrix(y_test, y_pred_if_best))

Best parameters for Isolation Forest: {'n_estimators': 50, 'max_samples': 'auto', 'max_features': 0.9, 'contamination': 0.06}
Tuned Isolation Forest:
Accuracy: 0.47284014485256076
              precision    recall  f1-score   support

           0       0.49      0.91      0.63      2899
           1       0.27      0.03      0.06      2900

    accuracy                           0.47      5799
   macro avg       0.38      0.47      0.34      5799
weighted avg       0.38      0.47      0.34      5799

[[2652  247]
 [2810   90]]


**Tuned Isolation Forest Results**
The tuned Isolation Forest model has an accuracy of 46.66%, which is slightly lower than the initial accuracy of 48.25%. Despite the tuning, the model's performance did not improve significantly in identifying fraudulent claims.

### Model 2: Gradient Boosting Model

This is an ensemble technique that builds models sequentially to correct the errors of previous models.

In [37]:
from sklearn.ensemble import GradientBoostingClassifier

# Define the Gradient Boosting model
gb_model = GradientBoostingClassifier(random_state=42)

# Fit the model to the training data
gb_model.fit(X_train, y_train)

# Predict on the test data
y_pred_gb = gb_model.predict(X_test)

print("Gradient Boosting:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gb)}")
print(classification_report(y_test, y_pred_gb))
print(confusion_matrix(y_test, y_pred_gb))

Gradient Boosting:
Accuracy: 0.804276599413692
              precision    recall  f1-score   support

           0       0.92      0.67      0.77      2899
           1       0.74      0.94      0.83      2900

    accuracy                           0.80      5799
   macro avg       0.83      0.80      0.80      5799
weighted avg       0.83      0.80      0.80      5799

[[1934  965]
 [ 170 2730]]


The **Gradient Boosting model** achieved an accuracy of 80.43%, showing it performs well in identifying both legitimate and fraudulent claims.

### Hyperparameter Tuning for Gradient Boosting Model

In [38]:
# Define the parameter grid
param_grid_gb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# Initialize GridSearchCV
grid_search_gb = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid_gb, cv=3, scoring='accuracy')

# Fit to the training data
grid_search_gb.fit(X_train, y_train)

# Best parameters and evaluation
print("Best parameters for Gradient Boosting:", grid_search_gb.best_params_)
best_gb = grid_search_gb.best_estimator_
y_pred_gb_best = best_gb.predict(X_test)

print("Tuned Gradient Boosting:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gb_best)}")
print(classification_report(y_test, y_pred_gb_best))
print(confusion_matrix(y_test, y_pred_gb_best))

Best parameters for Gradient Boosting: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 150}
Tuned Gradient Boosting:
Accuracy: 0.8544576651146749
              precision    recall  f1-score   support

           0       0.93      0.77      0.84      2899
           1       0.80      0.94      0.87      2900

    accuracy                           0.85      5799
   macro avg       0.86      0.85      0.85      5799
weighted avg       0.86      0.85      0.85      5799

[[2231  668]
 [ 176 2724]]


The **tuned Gradient Boosting model** has an accuracy of 85.45%, significantly improving its performance in identifying fraudulent claims.

### Model 3: Decision Tree Classifier

This is a simple, interpretable model that splits the data based on feature values to make decisions.

In [39]:
from sklearn.tree import DecisionTreeClassifier

# Define the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)

# Fit the model to the training data
dt_model.fit(X_train, y_train)

# Predict on the test data
y_pred_dt = dt_model.predict(X_test)

print("Decision Tree:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dt)}")
print(classification_report(y_test, y_pred_dt))
print(confusion_matrix(y_test, y_pred_dt))

Decision Tree:
Accuracy: 0.8542852215899293
              precision    recall  f1-score   support

           0       0.93      0.77      0.84      2899
           1       0.80      0.94      0.87      2900

    accuracy                           0.85      5799
   macro avg       0.86      0.85      0.85      5799
weighted avg       0.86      0.85      0.85      5799

[[2230  669]
 [ 176 2724]]


The **Decision Tree model** achieved an accuracy of 85.43%, showing it performs well in identifying both legitimate and fraudulent claims.

### Hyperparameter Tuning for Decision Tree Classifier

In [40]:
# Define the parameter grid
param_grid_dt = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV
grid_search_dt = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid_dt, cv=3, scoring='accuracy')

# Fit to the training data
grid_search_dt.fit(X_train, y_train)

# Best parameters and evaluation
print("Best parameters for Decision Tree:", grid_search_dt.best_params_)
best_dt = grid_search_dt.best_estimator_
y_pred_dt_best = best_dt.predict(X_test)

print("Tuned Decision Tree:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dt_best)}")
print(classification_report(y_test, y_pred_dt_best))
print(confusion_matrix(y_test, y_pred_dt_best))


Best parameters for Decision Tree: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Tuned Decision Tree:
Accuracy: 0.8542852215899293
              precision    recall  f1-score   support

           0       0.93      0.77      0.84      2899
           1       0.80      0.94      0.87      2900

    accuracy                           0.85      5799
   macro avg       0.86      0.85      0.85      5799
weighted avg       0.86      0.85      0.85      5799

[[2230  669]
 [ 176 2724]]


**Tuning Decision Tree Classifier Model** accuracy was unchanged.

### Models we can use 
* XGBoost: Highly efficient and often performs well in competitions. (Aaron)
* Random Forests: Robust against overfitting and handles imbalanced data well.  (Aaron)
* K-Nearest Neighbors (KNN): Simple and non-parametric, useful for small datasets. (Aaron)
* 
* Decision Trees: Useful for capturing non-linear relationships and easy to interpret. (Thai)
* Isolation Tree: It is a decision-tree-based algorithm originally developed for outlier detection. (Thai)
* Gradient Boosting Machines (GBM): Effective for capturing intricate patterns in data. (Thai)
* 
* Logistic Regression: Simple and interpretable, good for a baseline model. 
* Support Vector Machines (SVM): Suitable for high-dimensional spaces and can be adapted for imbalanced classes. 
* Naive Bayes: Good for text data or data with strong independence assumptions.
* Artificial Neural Networks (ANN): Flexible and powerful for capturing complex relationships.
* LightGBM: Known for its speed and efficiency, especially with large datasets.
* CatBoost: Handles categorical variables effectively and is robust with imbalanced data.
* AdaBoost: Combines weak learners to create a strong classifier, useful for boosting performance.
* Linear Discriminant Analysis (LDA): Can be used if the data distribution assumptions hold.
* Quadratic Discriminant Analysis (QDA): Useful for capturing more complex relationships than LDA.